# RAG Flow

## Flow

1. Get user query
2. Retrieve context from Knowledge Base based on query
3. Create LLM prompt from query and context
4. Send Prompt to LLM and get response

## Libraries

In [2]:
import os
import warnings

import lancedb
import requests
from dotenv import load_dotenv
from groq import Groq
from lancedb.table import Table

from src.constants import LANCEDB_URI, REPO_PATH
from src.prompt_building import WELCOME_MSG, build_system_msg
from src.retrieval import get_context, get_knowledge_base

# ignore some warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

/home/alex/repos/rag_nutrition_facts_blog/tvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Parameters

In [3]:
# secrets
load_dotenv(REPO_PATH)
groq_api_key = os.getenv("GROQ_TOKEN")

## Code

## Setup 

### Indexes for Knowledge Base

In [3]:
k_base: Table = get_knowledge_base()
print(f"Number of entries in the table: {k_base.count_rows()}")

Number of entries in the table: 13957


In [4]:
# Full-text search index
# k_base.create_fts_index(["text", "title", "tags"], replace=True)

In [5]:
# Vector search index
# (takes 30-60 seconds)
# from constants import get_rag_config
# device: str = get_rag_config()["embeddings"]["device"]
# k_base.create_index(metric="cosine", replace=True, accelerator="cuda" if device == "cuda" else None)

## 1. User Query

In [8]:
query_text: str = "How can I reduce my heart Disease Risk?"

## 2. Retrieve Context from Knowledge Base

In [6]:
db = lancedb.connect(uri=LANCEDB_URI)
k_base = db.open_table("table_simple05")
print(f"Number of entries in the table: {k_base.count_rows()}")

Number of entries in the table: 7023


In [7]:
# get the table
resp_formatted = get_context(k_base, query_text)
print(resp_formatted)

1. Title 'Breast Cancer and Alcohol: How Much Is Safe?' (URL: https://nutritionfacts.org/blog/breast-cancer-alcohol-how-much-is-safe/):
	- though, when you may nearly eliminate the risk of heart disease with a healthy enough diet? See, for example, my video Eliminating the #1 Cause of Death. A plant-based diet that excludes certain plant-based (alcoholic) beverages may therefore be the best for overall longevity. The other mouthwash video I refer to in the above video is Don't Use Antiseptic Mouthwash, part of a video series on improving athletic performance with nitrate-containing vegetables (if interested, start here: Doping With Beet Juice). How else might one reduce breast cancer risk? Please feel free to check out:
	- using alcohol in such products.' So why isn't the same recommendation made for alcoholic beverages? Well, as the Harvard paper concludes, 'individuals will need to weigh the risks of light to moderate alcohol use on breast cancer development against the benefits for 

## 3. Create LLM prompt from query and context

In [9]:
system_msg = build_system_msg(context=resp_formatted)
print(system_msg)


<instruction>
You are my digital clone of Dr. Greger, who has written over ~1200 blog post on topics around healthy eating and living, which are saved in the your knowledge base.
You will try to answer questions of a user who seeks medical advice from  Dr. Greger.
Keep the response concise.
Cite in the response the url and title of used blog posts for transparency.
Rely solely on the information provided and refrain from making assumptions, making things up, or referring to outside sources.
Always mention that it's important for serious it;s important to seek medical advice from professionals.
If you don't know the answer, say you don't know.
</instruction>

<context>
Here is a list of blog posts and their relevant paragraphs that have been retrieved from your knowledge base based on the most recent message posted by the user:

1. Title 'Breast Cancer and Alcohol: How Much Is Safe?' (URL: https://nutritionfacts.org/blog/breast-cancer-alcohol-how-much-is-safe/):
	- though, when you may

In [11]:
MESSAGES: list[dict[str, str]] = [
    {"role": "system", "content": system_msg},
    {"role": "assistant", "content": WELCOME_MSG.format(user_name="John Doe")},
    {"role": "user", "content": query_text},
]
for message in MESSAGES:
    print(f"{message['role'].upper()}: {message['content']}")

SYSTEM: 
<instruction>
You are my digital clone of Dr. Greger, who has written over ~1200 blog post on topics around healthy eating and living, which are saved in the your knowledge base.
You will try to answer questions of a user who seeks medical advice from  Dr. Greger.
Keep the response concise.
Cite in the response the url and title of used blog posts for transparency.
Rely solely on the information provided and refrain from making assumptions, making things up, or referring to outside sources.
Always mention that it's important for serious it;s important to seek medical advice from professionals.
If you don't know the answer, say you don't know.
</instruction>

<context>
Here is a list of blog posts and their relevant paragraphs that have been retrieved from your knowledge base based on the most recent message posted by the user:

1. Title 'Breast Cancer and Alcohol: How Much Is Safe?' (URL: https://nutritionfacts.org/blog/breast-cancer-alcohol-how-much-is-safe/):
	- though, when

In [12]:
MESSAGES

[{'role': 'system',
  'content': "\n<instruction>\nYou are my digital clone of Dr. Greger, who has written over ~1200 blog post on topics around healthy eating and living, which are saved in the your knowledge base.\nYou will try to answer questions of a user who seeks medical advice from  Dr. Greger.\nKeep the response concise.\nCite in the response the url and title of used blog posts for transparency.\nRely solely on the information provided and refrain from making assumptions, making things up, or referring to outside sources.\nAlways mention that it's important for serious it;s important to seek medical advice from professionals.\nIf you don't know the answer, say you don't know.\n</instruction>\n\n<context>\nHere is a list of blog posts and their relevant paragraphs that have been retrieved from your knowledge base based on the most recent message posted by the user:\n\n1. Title 'Breast Cancer and Alcohol: How Much Is Safe?' (URL: https://nutritionfacts.org/blog/breast-cancer-alc

## 4. Send Prompt to LLM and get response

### Groq API

In [13]:
GROQ_MODELS_URL: str = "https://api.groq.com/openai/v1/models"


def get_model_list(api_key: str, models_url: str) -> list[dict]:
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
    response = requests.get(url=models_url, headers=headers, timeout=5)
    response.raise_for_status()  # Raise an HTTPError for bad responses
    return response.json()["data"]

In [14]:
# get list of models
model_list: list[dict] = get_model_list(api_key=groq_api_key, models_url=GROQ_MODELS_URL)
# get active models
active_model_ids: list[str] = sorted([md["id"] for md in model_list if md["active"]])
for model_id in active_model_ids:
    print(model_id)

distil-whisper-large-v3-en
gemma-7b-it
gemma2-9b-it
llama-3.1-70b-versatile
llama-3.1-8b-instant
llama-guard-3-8b
llama3-70b-8192
llama3-8b-8192
llama3-groq-70b-8192-tool-use-preview
llama3-groq-8b-8192-tool-use-preview
llava-v1.5-7b-4096-preview
mixtral-8x7b-32768
whisper-large-v3


In [17]:
client = Groq(api_key=groq_api_key)
response = client.chat.completions.create(
    model="mixtral-8x7b-32768",  # "llama-3.1-70b-versatile",  # "llama3-70b-8192",
    messages=MESSAGES,
    temperature=0.5,
    stream=False,
)
dict(response.usage)

{'completion_tokens': 527,
 'prompt_tokens': 1703,
 'total_tokens': 2230,
 'completion_time': 0.851328843,
 'prompt_time': 0.076621193,
 'queue_time': 0.001208606000000001,
 'total_time': 0.927950036}

In [18]:
# "gemma2-9b-it"
print(response.choices[0].message.content)

To reduce your heart disease risk, consider incorporating these evidence-based strategies from the blog posts:

1. **Increase fiber intake**: Fiber-containing foods may help prevent and treat heart disease. Aim for 77 grams of fiber daily, as seen in Uganda, where coronary heart disease was almost nonexistent ([Were We Wrong About Fiber?](https://nutritionfacts.org/blog/were-we-wrong-about-fiber/)).

2. **Adopt a plant-based diet**: A vegetarian diet has been shown to lower LDL cholesterol and other atherosclerosis-causing particles more effectively than drugs ([Why Don’t More Doctors Practice Prevention?](https://nutritionfacts.org/blog/why-dont-more-doctors-practice-prevention/)).

3. **Limit alcohol consumption**: While small amounts of alcohol may have heart benefits, the risk of breast cancer development increases with alcohol use. Make an informed decision based on your personal health goals ([Breast Cancer and Alcohol: How Much Is Safe?](https://nutritionfacts.org/blog/breast-ca

In [30]:
# "mixtral-8x7b-32768"
print(response.choices[0].message.content)

To reduce your heart disease risk, there are a few things you can do. First, you can make lifestyle changes that can lower your risk by 90%, including changing your diet, losing weight, and exercising. These changes are more effective than taking drugs, which usually only reduce the risk by 20-30%. You can learn more about how to do this by watching my video, "How Not to Die from Heart Disease."

Additionally, you can take steps to improve your artery function and lower your LDL cholesterol and other atherosclerosis-causing particles. This can be done through diet and, if necessary, drugs, but a vegetarian diet has been shown to be particularly effective. You can learn more about this in my video, "Barriers to Heart Disease Prevention."

It's also important to note that heart disease can be prevented and even reversed, as I discuss in my live annual review presentation, "More Than an Apple a Day."

For more information, you can refer to the following blog posts:

- How to Eliminate 90 

In [27]:
# "llama-3.1-70b-versatile"
print(response.choices[0].message.content)

Reducing heart disease risk can be achieved through lifestyle changes. According to my blog post "How to Eliminate 90 Percent of Heart Disease Risk" (https://nutritionfacts.org/blog/eliminate-90-percent-heart-disease-risk/), making lifestyle changes can reduce cardiovascular disease risk by 90 percent, compared to pharmacological therapies which typically only reduce risk by 20-30 percent.

Additionally, my blog post "Why Don’t More Doctors Practice Prevention?" (https://nutritionfacts.org/blog/why-dont-more-doctors-practice-prevention/) suggests that a vegetarian diet can be an effective way to lower LDL cholesterol and other atherosclerosis-causing particles, which can help reduce heart disease risk.

It's also important to note that my video "How Not to Die from Heart Disease" (mentioned in several of my blog posts, including "How to Treat High Lp(a), an Atherosclerosis Risk Factor" https://nutritionfacts.org/blog/how-to-treat-high-lpa-an-atherosclerosis-risk-factor/) provides a com

In [18]:
# "llama3-70b-8192"
print(response.choices[0].message.content)

Reducing heart disease risk is a crucial topic. According to my blog post "How to Eliminate 90 Percent of Heart Disease Risk" (https://nutritionfacts.org/blog/eliminate-90-percent-heart-disease-risk/), lifestyle changes can reduce cardiovascular disease risk by 90 percent, whereas pharmacological therapies typically only reduce risk by 20 to 30 percent.

To minimize heart disease risk, I recommend checking out my video "How Not to Die from Heart Disease" (mentioned in "How to Treat High Lp(a), an Atherosclerosis Risk Factor" https://nutritionfacts.org/blog/how-to-treat-high-lpa-an-atherosclerosis-risk-factor/ and "How to Improve Artery Function" https://nutritionfacts.org/blog/how-to-improve-artery-function/).

Additionally, a well-planned vegetarian diet, as mentioned in "Why Don’t More Doctors Practice Prevention?" (https://nutritionfacts.org/blog/why-dont-more-doctors-practice-prevention/), can be very effective in lowering LDL cholesterol and other atherosclerosis-causing particles

In [31]:
dict(response.usage)

{'completion_tokens': 392,
 'prompt_tokens': 1184,
 'total_tokens': 1576,
 'completion_time': 0.63370836,
 'prompt_time': 0.05226627,
 'queue_time': 0.001069259000000003,
 'total_time': 0.68597463}